## RDKit Tutorial, October 2020, Bender Group

In this tutorial we will cover basic functionalities of RDKit and Jupyter Notebooks.

You should have followed the instructions outlined in the "Getting Started with RDKit.pdf" file.

First, click on this "cell" and see that it is a "Markdown" cell. This means that here you can write text to explain and document your code.

Now, click on the next cell and notice it says "Code"

In [ ]:
# Import packages
# (to annotate in a "code" cell you must use a hash symbol)
import pandas as pd
import numpy as np

With Jupyter notebooks it is easy to perform exploratory analysis and test your code. For example, say you are creating a function to add 2 to an input number...

In [ ]:
# declare new function
def add_two(num):
    print(num+3)

In [ ]:
# apply the function
add_two(5)

Hmm, looks like there's something wrong here. Instead of having to diagnose the issue, attempt to fix it and run the entire script again, we can simply modify and run just a couple of cells, separately from the rest of the notebook.

In [ ]:
# declare new function
def add_two(num):
    print(num+2)

In [ ]:
# apply the function
add_two(5)

That was easy!

With Jupyter notebooks you can also display dataframes to ensure that everything is as it should be

In [ ]:
# create a df
df = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
                   columns=['a', 'b', 'c'])

In [ ]:
# display df
df

In [ ]:
# display first two rows of df
df.head(2)

You can also display images and do a bunch of other cool stuff, but we will explore these as we go through the RDKit tutorial

### Getting Started With RDKit

All material is from here https://www.rdkit.org/docs/GettingStartedInPython.html which gives a nice overview of RDKit functionalities. Thus, I give full credit to the RDKit authors and don't try to claim this tutorial as my own! (I just put it into a Jupyter notebook...)

Most of the RDKit functionality is in the module rdkit.Chem. Let's import it!

In [ ]:
from rdkit import Chem

You can construct molecule objects using different approaches

In [ ]:
m = Chem.MolFromSmiles('Cc1ccccc1')
m

In [ ]:
m = Chem.MolFromMolFile('input.mol') # .mol file
m

In [ ]:
stringWithMolData=open('input.mol','r').read()
m = Chem.MolFromMolBlock(stringWithMolData)
m

If the import does not work, you will return None (good for error catching), e.g.:

In [ ]:
m = Chem.MolFromMolFile('invalid.mol')
print(m is None)

The error messages are often very informative:

In [ ]:
m2 = Chem.MolFromSmiles('c1cc1')

### Reading sets of molecules
Reading groups of molecules is done using a Supplier:

In [ ]:
suppl = Chem.SDMolSupplier('5ht3ligs.sdf') # sdf
for mol in suppl:
    if mol is None: continue # catch erroneous mols
    print(mol.GetNumAtoms()) # print the number of atms in each mol object

Lists of molecules can also be generated:

In [ ]:
mols = [x for x in suppl]
len(mols)

You can also directly access the suppl object

In [ ]:
suppl[0].GetNumAtoms()

### Writing molecules
You can also convert molecule objects back to text

In [ ]:
m = Chem.MolFromMolFile("input.mol") # this provides CANONICAL smiles
Chem.MolToSmiles(m)

You can also kekulize molecules before converting back to SMILES

In [ ]:
Chem.Kekulize(m)
Chem.MolToSmiles(m,kekuleSmiles=True) # NOT canonical

You can also save as MDL MOL blocks

In [ ]:
m.SetProp("_Name","test") # you can set a name so that it shows up in the MOL block
print(Chem.MolToMolBlock(m))

### Generating coordinates
It is possible to generate 2D and 3D coordinates too, with the 'AllChem' module

In [ ]:
#2D
from rdkit.Chem import AllChem
AllChem.Compute2DCoords(m)
print(Chem.MolToMolBlock(m))

In [ ]:
#3D
m2 = Chem.AddHs(m) # add hydrogens first
AllChem.EmbedMolecule(m2,randomSeed=0xf00d) # random seed for reproducibility
print(Chem.MolToMolBlock(m2))

In [ ]:
# remove hydrogens
m3 = Chem.RemoveHs(m2)
print(Chem.MolToMolBlock(m3))

In [ ]:
# write to file
print(Chem.MolToMolBlock(m3),file=open('test.mol','w+'))

### Creating 3D conformations
RDKit can also generate 3D conformations and do some energy minimisation

In [ ]:
m = Chem.MolFromSmiles('C1CCC1OC') # first read in a mol object
m2 = Chem.AddHs(m) # add Hs
AllChem.EmbedMolecule(m2) # make 3D molecule (ETKDG method)
res = AllChem.MMFFOptimizeMoleculeConfs(m2) #MMFF94 optimisation
print(res) # (not_converted,energy) if not_converged=0, then the minimsisation converged

In [ ]:
m2

In [ ]:
# To create multiple conformers, run ETKDG e.g. 10 times
cids = AllChem.EmbedMultipleConfs(m2,numConfs=10)

In [ ]:
# Now you can align the conformers and see the RMS values between them
rmslist = []
AllChem.AlignMolConformers(m2,RMSlist=rmslist)
rmslist # RMS values between first conformer and all others

In [ ]:
# You can also specify RMS between two specific confomers
rms = AllChem.GetConformerRMS(m2,1,9,prealigned=True)
rms

### Writing sets of molecules
You can write multiple molecules, e.g. an SDF file, with the following:

In [ ]:
w = Chem.SDWriter('test.sdf')
for m in mols:
    w.write(m)

### Drawing Molecules
You can create images from molecules with the rdkit.Chem.Draw package

In [ ]:
suppl = Chem.SDMolSupplier('cdk2.sdf') # read in .sdf file
ms = [x for x in suppl if x is not None] # make into list of mol objects
for m in ms: tmp=AllChem.Compute2DCoords(m) # for each mol, compute the 2D coords
from rdkit.Chem import Draw # import package
Draw.MolToFile(ms[0],'cdk2_mol1.o.png') # draw mol file

It is even possible to create an image grid out of a set of molecules

In [ ]:
img=Draw.MolsToGridImage(ms[:8],molsPerRow=4,subImgSize=(200,200),legends=[x.GetProp("_Name") for x in ms[:8]])   
img.save('cdk2_molgrid.o.png')    

It would look even nicer if all molecules were aligned with a common core!

In [ ]:
p = Chem.MolFromSmiles('[nH]1cnc2cncnc21') # define substructure
subms = [x for x in ms if x.HasSubstructMatch(p)] # search for mol objects which have this particular substructure
print(len(subms)) # how many?
AllChem.Compute2DCoords(p) # compute 2D coords of the substructure core
for m in subms: AllChem.GenerateDepictionMatching2DStructure(m,p) # generate 2D structures aligned to this common core
img=Draw.MolsToGridImage(subms,molsPerRow=4,subImgSize=(200,200),legends=[x.GetProp("_Name") for x in subms])    
img.save('cdk2_molgrid.aligned.o.png')

Take a look and see the difference in the two pictures!
Not aligned:

<img src = "cdk2_molgrid.o.png">

Aligned:

<img src = 'cdk2_molgrid.aligned.o.png'>

### Substructure Searching
Substructure matching is done using query molecules from SMARTS

In [ ]:
m = Chem.MolFromSmiles('c1ccccc1O') # create mol object
patt = Chem.MolFromSmarts('ccO') # create SMARTS query
m.HasSubstructMatch(patt) # TRUE means it contains the substructure

In [ ]:
# You can also get the atom indices for the substruct:
m.GetSubstructMatch(patt)

In [ ]:
# display the match
m

Stereochemistry is NOT used by default in substructure searches

In [ ]:
m = Chem.MolFromSmiles('CC[C@H](F)Cl')
print(m.HasSubstructMatch(Chem.MolFromSmiles('C[C@H](F)Cl')))
print(m.HasSubstructMatch(Chem.MolFromSmiles('C[C@@H](F)Cl')))
print(m.HasSubstructMatch(Chem.MolFromSmiles('CC(F)Cl')))

Change this behaviour with the useChirality argument

In [ ]:
m.HasSubstructMatch(Chem.MolFromSmiles('C[C@H](F)Cl'),useChirality=True)

In [ ]:
m.HasSubstructMatch(Chem.MolFromSmiles('C[C@@H](F)Cl'),useChirality=True)

### Structure transformations
Simple modifications of compounds can also be carried out

In [ ]:
m = Chem.MolFromSmiles('CC(=O)O')
patt = Chem.MolFromSmarts('C(=O)[OH]')
rm = AllChem.DeleteSubstructs(m,patt) # DELETE substructure
Chem.MolToSmiles(rm)

In [ ]:
m # original

In [ ]:
patt # sub

In [ ]:
rm # removed

In [ ]:
repl = Chem.MolFromSmiles('OC') # replacement
patt = Chem.MolFromSmarts('[$(NC(=O))]') # pattern
m = Chem.MolFromSmiles('CC(=O)N') # query
rms = AllChem.ReplaceSubstructs(m,patt,repl) # replace patt in m with repl
Chem.MolToSmiles(rms[0])

In [ ]:
m # original

In [ ]:
rms[0] # replaced 

### Generate Murcko Scaffolds
Quickly get Murcko scaffolds with RDKit

In [ ]:
from rdkit.Chem.Scaffolds import MurckoScaffold # import package
cdk2mols = Chem.SDMolSupplier('cdk2.sdf') # load SDF file
m1 = cdk2mols[0] # pick out first mol
core = MurckoScaffold.GetScaffoldForMol(m1) # get Murcko Scaffold
Chem.MolToSmiles(core) # convert to smiles

In [ ]:
core

### Generate Morgan Fingerprints (ECFP4) and Calculate Similarity
Morgan fingerprints can also be generated, useful for calculating similarity between molecules

In [ ]:
from rdkit.Chem import AllChem # AllChem is needed
from rdkit import DataStructs # for similarity 
m1 = Chem.MolFromSmiles('Cc1ccccc1')
fp1 = AllChem.GetMorganFingerprint(m1,2) # radius 2
fp1 # by default, it is a sparse integer vector object

You can also generate the fingerprints as bit vectors, which is probably what you want:

In [ ]:
fp1 = AllChem.GetMorganFingerprintAsBitVect(m1,2,nBits=1024) # 1024-length ECFP4 fingerprints
fp1

These fingerprints can be used to compute Tanimoto similarities!

In [ ]:
m2 = Chem.MolFromSmiles('Cc1ncccc1')
fp2 = AllChem.GetMorganFingerprintAsBitVect(m2,2,nBits=1024)
DataStructs.TanimotoSimilarity(fp1,fp2)

To get the fingerprints as 0 and 1 for modelling, do the following:

In [ ]:
fp1.ToBitString()

In [ ]:
fp2.ToBitString()

You can also visualise the fingerprints by highlighting the atom environments which define a particular bit

In [ ]:
from rdkit.Chem import Draw # Draw package
mol = Chem.MolFromSmiles('c1ccccc1CC1CC1') # get mol object
bi = {} # initialise empty dict
fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, bitInfo=bi) # when generating fp, put bit info into dict
print(bi[872]) # get a particular bit, e.g. bit number 872
mfp2_svg = Draw.DrawMorganBit(mol, 872, bi, useSVG=True) # use this function to draw the bit
mfp2_svg # blue = central atom, yellow = aromatic atoms, grey = aliphatic ring atoms

In [ ]:
mol # see the full mol

### Picking Diverse Molecules Using Fingerprints

A common task is to pick a small subset of diverse molecules from a larger set. The RDKit provides a number of approaches for doing this in the rdkit.SimDivFilters module. The most efficient of these uses the MaxMin algorithm. 


1. Generate descriptors (e.g. Morgan fingerprints) for all the molecules, both any initial seeds plus those to pick from (the candidate pool). For large sets this will be slow, but cannot be avoided.
2. If there are no initial seeds select an initial molecule at random from the candidate pool and it becomes the sole member of the picked set.
3. From the molecules in the candidate pool find the one that has the maximum value for its minimum distance to molecules in the picked set (hence the MaxMin name), calculating and recording the distances as required. This molecule is the most distant one to those already picked so is transferred to the picked set.
4. Iterate back to step 3 until your are complete (e.g. you have picked the required number of molecules).

Start by reading in a set of molecules and generating Morgan fingerprints:

In [ ]:
# import
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprint
from rdkit import DataStructs
from rdkit.SimDivFilters.rdSimDivPickers import MaxMinPicker

# load .sdf file
ms = [x for x in Chem.SDMolSupplier('actives_5ht3.sdf')]

# remove None
while ms.count(None): ms.remove(None)
    
# generate fingerprints
fps = [GetMorganFingerprint(x,2) for x in ms] # ECFP4
nfps = len(fps)
nfps # 180 compounds in total

The algorithm requires a function to calculate distances between objects, we’ll do that using TanimotoSimilarity:

In [ ]:
def distij(i,j,fps=fps):
    return 1-DataStructs.TanimotoSimilarity(fps[i],fps[j]) # 1 - Similarity = Distance!

Now create a picker and grab a set of 10 diverse molecules:

In [ ]:
picker = MaxMinPicker() # use the MaxMin picker
pickIndices = picker.LazyPick(distij,nfps,10,seed=23)#we are telling it to use Tc, pick 10 diverse molecules
list(pickIndices)

This gives us the Indices, but to get the molecules themselves we do this:

In [ ]:
picks = [ms[x] for x in pickIndices]
picks # now we have 10 mol objects of diverse molecules!

### Calculate Descriptors
With RDKit you can also calculate a myriad of descriptors

In [ ]:
from rdkit.Chem import Descriptors
m = Chem.MolFromSmiles('c1ccccc1C(=O)O')
# TPSA
Descriptors.TPSA(m)

In [ ]:
#LogP
Descriptors.MolLogP(m)

List of all available descriptors can be found here: https://www.rdkit.org/docs/GettingStartedInPython.html#list-of-available-descriptors

# Fin.